# Pierre VAMBENEPE (Feb-2021)
# Deutsche Boerse A7 usage example for equity options


**Identifying clusters of trades with similar characteristics in order to detect trades dynamic**

Abstract :
It is often difficult to know what's going on in the Equity Options market.

But it is especially important to detect even small trading patterns because it's easy to miss new information
which would be relevant for options pricing. Trades can reveal that something is going on.

One important aspect of a trade analysis is the interest behind it. Whether the aggressor was the buyer or the seller,
it  doesn't tell us who was actually crossing the spread to make the trade happen (if any). To figure out if the interest
was the buyer or the seller and how aggressive it was, we will first calibrate a volatility surface in order to get
a theoretical bid and offer price, undisturbed by local (ie. strike specific) microstructure action. The aggressivity
parameter is defined as such :

aggressivity = min(1, max(-1, (traded_price - mid_theo) / half_theo_spread))

NB : aggressivity is negative for selling interest and positive for buying ones.

This theoretical price will then be used in conjunction with the vega of the trade to determine the intensity of a
a trade. It is defined as :

intensity = vega * aggressivity

This metrics among others will then be used to identify clusters of similar trades. These clusters will in turn be sorted
by vega_aggressivity in order to be able to report on the most remarkable trade action in the period.


In [ ]:
#First some global setting :

#NB: This program has been developed on Pycharm Professional for windows 10
#The Python exec is installed as a specific miniconda env running Python 3.6 with the following extensions

# Note on Pycharm : it was for me necessary to install the following kernel on conda in order to let Jyputer Notebook execute :
# conda activate myenv
# python -m ipykernel install --user --name myenv --display-name "Python (myenv)
# cf https://youtrack.jetbrains.com/issue/PY-36913

# Here are the needed external library
import numpy as np
import pandas as pd
import QuantLib as ql  #complete set of tools for derivatives pricing and calendar manipulation with exchanges holidays
import math
import datetime
import matplotlib.pyplot as plt
import requests   #in order to retrieve data from A7
import warnings

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 30)

#indicate here the folders where you want the quotes and trades data (folder1) and the calibration result with "fleshed" trades (folder 2)
folder1 = '/Users/pvamb/A7/processed'
folder2 = '/Users/pvamb/A7/parameters'


In [ ]:
# We are now going to import specific libraries made available in this git

from DateAndTime import DateAndTime
# uses QuantLib to calculate numbers of business day between dates and generate expi dates

from PricingAndCalibration import Pricing
# uses Quantlib to price European and American options with continuous dividend yield and the associated greeks

from PricingAndCalibration import FittingSpline
# uses scipy UnivariateSpline to fit a 2nd degree spline through the implicit vol of bid and ask quotes

from TradeFlesh import TradeFlesh
# enrich trades description with vega_aggressivity measure and provide graphic representations of trading patterns

from Clustering import Clustering
# uses sklearn AgglomerativeClustering in order to identify clusters of similar trades, potzentially steming from the same agent

**We will first retrieve the orderbook data from A7**

In [ ]:
from SetUp import *
from DateAndTime import DateAndTime

DT = DateAndTime(from_date = '2021-01-04', until_date = '2021-01-05')

owner = 'pierrev'
proxies = {
    "http": "",  # Enter http Proxy if needed",
    "https": ""  # Enter https Proxy if needed",
}

API_TOKEN = "Bearer " + "place your A7 token key here"

# algo = 'minsize_level'
algo = 'top_level'


udl = 'DAI'
isin = 'DE0007100000'
min_lots = 37 #(1/20th of block trade size)
# min_lots can be set to whatever level but if you want to refer to block trade size, here is where to find it :
# https://www.eurex.com/resource/blob/1637254/e8d6c53461a06ae89c7f38c2222da612/data/contract_specifications_en_ab-2021_01_18.pdf
# page 246-xxx

#filter settings to speed up the process
# for 1 year maturity option with an adjustment in sqrt(T)
moneyness_range_call = (-0.4, 0.7)
moneyness_range_put = (-0.7, 0.4)

a = datetime.datetime.now()  #time check

try:
    df_orderbook = pd.read_pickle(folder1 + '/Quotes_' + udl + '.pkl')
except:
    df_orderbook = pd.DataFrame()

try:
    df_trades = pd.read_pickle(folder1 + '/Trades_' + udl + '.pkl')
except:
    df_trades = pd.DataFrame()


done_already1 = [elt.strftime('%Y%m%d') for elt in set([elt.date() for elt in df_orderbook.index])]
done_already2 = [elt.strftime('%Y%m%d') for elt in set([elt.date() for elt in df_trades.index])]

for reference_date in [elt for elt in DT.dates_list if (elt not in done_already1 + done_already2) and (pd.Timestamp(elt) > df_orderbook.index.min())]:
    print(reference_date)

    try:
        #stock
        url = 'https://a7.deutsche-boerse.com/api/v1/rdi/XETR/{}?mode=detailed'.format(reference_date)
        r = requests.get(url=url,  headers={'Authorization': API_TOKEN}, proxies = proxies)
        res = r.json()

        lst_ms = np.array([x['MarketSegment'] for x in res['MarketSegments']])
        indx = np.where(lst_ms==isin)[0][0]
        segmentIDudl = res['MarketSegments'][indx]['MarketSegmentID']
        print('Market Segment for the underlying {} :: {}'.format(udl, str(segmentIDudl)))

        url = 'https://a7.deutsche-boerse.com/api/v1/rdi/XETR/{}/{}?mode=detailed'.format(reference_date, segmentIDudl)
        r = requests.get(url=url, headers={'Authorization': API_TOKEN}, proxies = proxies)
        res_u = r.json()
        security = res_u['Securities'][0]


        #Options

        url = 'https://a7.deutsche-boerse.com/api/v1/rdi/XEUR/{}?mode=detailed'.format(reference_date)
        r = requests.get(url = url,  headers={'Authorization': API_TOKEN}, proxies = proxies)
        res = r.json()

        lst_ms = np.array([x['MarketSegment'] for x in res['MarketSegments']])
        indx = np.where(lst_ms==udl)[0][0]
        segmentIDopt = res['MarketSegments'][indx]['MarketSegmentID']
        print('Market Segment for options on {} :: {}'.format(udl, str(segmentIDopt)))

        url = 'https://a7.deutsche-boerse.com/api/v1/rdi/XEUR/{}/{}?mode=detailed'.format(reference_date, segmentIDopt)
        r = requests.get(url = url,  headers={'Authorization': API_TOKEN}, proxies = proxies)
        res_i = r.json()

        b = datetime.datetime.now()
        print(b-a)
        a = b


        # retrieves quotes

        selected_fields = ['SecurityDesc', 'SecurityID']
        selected_fields_desc = ['PutOrCall', 'StrikePrice', 'ContractMultiplier', 'ExerciseStyle']

        raw = pd.DataFrame()
        matulist = sorted(list(set([str(elt['MaturityDate']) for elt in res_i['Securities'] if elt['MaturityDate'] != None])))
        for matu in ['UDL'] + matulist:
            print('    ' + matu)

            df = pd.DataFrame(columns=['SegmentID'] + selected_fields + selected_fields_desc)

            if matu == 'UDL':
                df.loc[0] = [segmentIDudl, security['SecurityDesc'], security['SecurityID'], 'S', None, 1, None]
                df['in_range'] = True
            else:
                i = 0
                for x in res_i['Securities']:
                    if (str(x['MaturityDate']) == matu) and (x['SecurityType'] == 'OPT'):
                        df.loc[i] = [segmentIDopt] + [x[elt] for elt in selected_fields] + \
                                    [x['DerivativesDescriptorGroup']['SimpleInstrumentDescriptorGroup'][elt] for elt in selected_fields_desc]
                        i += 1

                df.sort_values(by=['StrikePrice', 'PutOrCall'], ascending = [True, True], inplace=True)

                TTM = DT.time_between(pd.Timestamp(reference_date), pd.Timestamp(matu))
                df['moneyness_T'] = df.apply(lambda opt: math.log(opt.StrikePrice / FVU) / (max(3.0 / 12.0, TTM) ** 0.5), axis='columns')
                # the forward ratio is unknown at this stage so we take a high dividend rate of 8% as instead, hence the 0.92
                df['moneyness_T_w_div'] = df.apply(lambda opt: math.log(opt.StrikePrice / FVU*0.92) / (max(3.0 / 12.0, TTM) ** 0.5), axis='columns')
                df['in_range'] = df.apply(lambda opt: (opt.moneyness_T_w_div > moneyness_range_call[0]) and (opt.moneyness_T < moneyness_range_call[1])  \
                        if opt.PutOrCall == '1' else \
                        (opt.moneyness_T_w_div > moneyness_range_put[0]) and (opt.moneyness_T < moneyness_range_put[1]), axis='columns')

                df = df.loc[df.in_range]

            for index, opt in df.iterrows():
                # print(opt)

                if opt['PutOrCall'] == 'S':
                    market = 'XETR'
                    url = 'https://a7.deutsche-boerse.com/api/v1/algo/{}/top_level/'.format(owner)
                    url = url+"run?marketId={}&date={}&marketSegmentId={}&securityId={}".format(market, reference_date, opt['SegmentID'], opt['SecurityID'])

                else:
                    market = 'XEUR'
                    if algo == 'top_level':
                        url = 'https://a7.deutsche-boerse.com/api/v1/algo/{}/top_level/'.format(owner)
                        url = url+"run?marketId={}&date={}&marketSegmentId={}&securityId={}".format(market, reference_date, opt['SegmentID'], opt['SecurityID'])
                    elif algo == 'minsize_level':
                        url = 'https://a7.deutsche-boerse.com/api/v1/algo/{}/minsize_level/'.format(owner)
                        url = url+"run?marketId={}&date={}&marketSegmentId={}&securityId={}&aggr=1&min_lots={}".format(market, reference_date, opt['SegmentID'], opt['SecurityID'], min_lots)

                r = requests.get(url=url,  headers={'Authorization': API_TOKEN}, proxies = proxies)
                res = r.json()

                if type(res) == list:
                    bid_ask_sampled = {}
                    for i, bidask in enumerate(['bid', 'ask']):
                            df_price = pd.DataFrame(index=res[0]['series'][i]['content']['ts'])
                            df_price = df_price.assign(pv=res[0]['series'][i]['content']['price'])

                            df_price = df_price.dropna()
                            if df_price.shape[0] > 0:
                                df_price['pv'] = df_price['pv'].astype(float)/1e3
                                df_price.columns = [bidask]
                                df_price.index = df_price.index.astype(np.int64)
                                df_price.index = pd.to_datetime(df_price.index)

                                for elt in selected_fields_desc:
                                    df_price[elt] = opt[elt]
                                df_price['matu'] = matu

                                index = pd.date_range(df_price.index[0].round('T'), df_price.index[-1], freq='1T')
                                df_price = df_price.reindex(index, method='ffill')

                                bid_ask_sampled[bidask] = df_price

                    if len(bid_ask_sampled) == 2:
                        df_opt = pd.merge(bid_ask_sampled['bid'][['bid']], bid_ask_sampled['ask'], how='inner', left_index=True, right_index=True)
                        if opt['PutOrCall'] == 'S':
                            FVU = (df_opt.bid.median() + df_opt.ask.median())/2
                        # df_opt['matu'] = matu
                        df_orderbook = df_orderbook.append(df_opt)

        b = datetime.datetime.now()
        print(b-a)
        a = b

        df_orderbook.to_pickle(folder1 + '/Quotes_' + udl + '.pkl')

        b = datetime.datetime.now()
        print(b-a)
        a = b


        #retrieve trades

        selected_fields = ['SecurityDesc', 'SecurityID']
        selected_fields_desc = ['PutOrCall', 'StrikePrice', 'ContractMultiplier', 'ExerciseStyle']

        for matu in matulist:

            df = pd.DataFrame(columns=['SegmentID'] + selected_fields + selected_fields_desc)
            i = 0
            for x in res_i['Securities']:
                if (str(x['MaturityDate']) == matu) and (x['SecurityType'] == 'OPT'):
                    df.loc[i] = [segmentIDopt] + [x[elt] for elt in selected_fields] + \
                                [x['DerivativesDescriptorGroup']['SimpleInstrumentDescriptorGroup'][elt] for elt in selected_fields_desc]
                    i += 1

            for index, opt in df.iterrows():
                # print(opt['SecurityDesc'])

                url = 'https://a7.deutsche-boerse.com/api/v1/algo/{}/trades_PVA/'.format(owner)

                market = 'XEUR'
                url = url+"run?marketId={}&date={}&marketSegmentId={}&securityId={}".format(market, reference_date, opt['SegmentID'], opt['SecurityID'])
                r = requests.get(url=url,  headers={'Authorization': API_TOKEN}, proxies = proxies)
                res = r.json()

                if (type(res) == list) and (len(res[0]['series'][0]['content']['time'])>0):
                    df_opt = pd.DataFrame.from_dict(res[0]['series'][0]['content'])
                    df_opt.index = df_opt.index.astype(np.int64)
                    df_opt.index = pd.to_datetime(df_opt.index)
                    for field in ['time', 'priots', 'bidentry', 'askentry']:
                        df_opt[field] = df_opt[field].astype(np.int64)
                        df_opt[field] = pd.to_datetime(df_opt[field])
                    df_opt.set_index('time', inplace=True)
                    # df_field = df_field.dropna()

                    df_opt[selected_fields_desc] = opt[selected_fields_desc]

                    df_opt['matu'] = matu
                    df_opt['SegmentID'] = opt['SegmentID']
                    df_opt['SecurityID'] = opt['SecurityID']
                    df_trades = df_trades.append(df_opt)

        df_trades.to_pickle(folder1 + '/Trades_' + udl + '.pkl')

        b = datetime.datetime.now()
        print(b-a)
        a = b

    except:
        print('fail for :' + reference_date)


    udl = 'DBK'
    FS = FittingSpline(udl)
    FS.fit_all()

    done_already = [elt.strftime('%Y%m%d') for elt in set([elt.date() for elt in self.df_params.index.get_level_values(0)])]

    for reference_date in [elt for elt in self.dates_list if (elt not in done_already)]:
        print(reference_date)
        matulist = [elt for elt in self.get_matu_list(reference_date) if elt != reference_date]

        for matu in matulist:
            print('   ' + matu)

            #ini_day intializies the dataframe and sets the starting implicit vol flat at 30% if it is the first day
            FS.ini_day(reference_date, matu)

            #fit_day starts a process of fitting the vol curve every 5 minutes allong with the Fwd ratio (dividend + repo)
            FS.fit_day()

        self.df_params.to_pickle(folder2 + '/Params_' + self.udl + '.pkl')

Congratulations, you have created a parameters dataframe with the fitted spline curve for the bid and ask implicit vol


In [ ]:
#Let's now graph what we have done :
FS.graph("20210105", "20210319")

# We will now use the calibration to determine the aggressivity factor for each trade :
TF = TradeFlesh(udl)
TF.pct_aggressivity()
# The result is saved in the FleshedTrades file in folder2

# ...and get a view of the trades to .
TF.graph_aggressivity('20210105')

In [ ]:
# We will now calculate the intensity of the trades then aggregate them over 1 minutes intervals.
TF.get_intensity()
# We can now observe the meaningfull trades, that is those which are large and for which one party has crossed a significant part of the spread.
TF.graph_sensitivity('vega', '20190710')

In [ ]:
# A high intensity trades may be meaningfull but a bunch of similar ones with similar characteritic are even more so.
# They may may point to an agent who is either informed or with a large size to trade and help anticipate future parameters shift.
C = Clustering(udl)
C.prepare_data(with_graph = True)
# the first graph shows the hierarchical clustering. We define a cluster as a set whose max distance is less than 5 times it's distance to to next cluster
# The distance refered to here is a

C.display_clusters(5)
TF.graph_aggressivity('20210105', C.trades(0))